# Quantitative Value Strategy

"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).
For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests 
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_excel('sp_500_stocks.xlsx')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!
We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'L/T LMGS NK QN(O)ACASLASGTEBRDAEE',
 'calculationPrice': 'close',
 'open': None,
 'openTime': None,
 'openSource': 'focaifil',
 'close': None,
 'closeTime': None,
 'closeSource': 'oifaclfi',
 'high': None,
 'highTime': None,
 'highSource': None,
 'low': None,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 134.68,
 'latestSource': 'Close',
 'latestTime': 'December 30, 2020',
 'latestUpdate': 1685930342748,
 'latestVolume': None,
 'iexRealtimePrice': 135.79,
 'iexRealtimeSize': 31,
 'iexLastUpdated': 1628456727978,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 135.77,
 'previousVolume': 121328071,
 'change': -1.17,
 'changePercent': -0.00888,
 'volume': None,
 'iexMarketPercent': 0.0025456768363622756,
 'iexVolume': 1896,
 

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.
Here is an example of how to parse the metric from our API call:

In [4]:
pe_ratio = data['peRatio']
pe_ratio

41.9

## Executing A Batch API Call & Building Our DataFrame
Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
def chunks(lst, n):
    '''Yield successive n-sized chunks from lst/'''
    for i in range(0, len(lst), n):
        yield lst[i:i +n]

symbol_group = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_group)):
    symbol_strings.append(','.join(symbol_group[i]))

columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns=columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series([symbol, 
                   data[symbol]['quote']['latestPrice'],
                   data[symbol]['quote']['peRatio'],
                   'N/A'], 
                  index = columns), 
                  ignore_index = True)

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,121.98,55.57,N/A
1,AAL,16.18,-1.16,N/A
2,AAP,162.24,23.00,N/A
3,AAPL,136.38,41.93,N/A
4,ABBV,105.75,23.11,N/A
...,...,...,...,...
500,YUM,110.54,32.91,N/A
501,ZBH,155.92,-212.06,N/A
502,ZBRA,394.49,44.02,N/A
503,ZION,43.79,19.23,N/A


## Removing Glamour Stocks
The opposite of a "value stock" is a "glamour stock".
Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace= True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace= True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,37.25,2.27,N/A
1,NLOK,20.86,4.14,N/A
2,UNM,22.76,4.91,N/A
3,AIV,5.24,5.08,N/A
4,BIO,579.27,5.11,N/A
5,KIM,15.00,6.72,N/A
6,ALL,108.22,7.59,N/A
7,MET,47.70,7.63,N/A
8,EBAY,50.55,7.95,N/A
9,BIIB,255.40,8.25,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy.

To do this, we will use the portfolio_input function that we created in our momentum project.


In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:10000000
10000000


In [9]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,37.25,2.27,5369
1,NLOK,20.86,4.14,9587
2,UNM,22.76,4.91,8787
3,AIV,5.24,5.08,38167
4,BIO,579.27,5.11,345
5,KIM,15.00,6.72,13333
6,ALL,108.22,7.59,1848
7,MET,47.70,7.63,4192
8,EBAY,50.55,7.95,3956
9,BIIB,255.40,8.25,783


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a composite basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:
- Price-to-earnings ratio
- Price-to-book ratio
- Price-to-sales ratio
- Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
- Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [10]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation rv often. It stands for robust value, which is what we'll call this sophisticated strategy moving forward.

In [11]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

In [12]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,118.32,N/A,53.76,N/A,7.2600,N/A,7,N/A,33.889454,N/A,13.187126,N/A,N/A
1,AAL,16.95,N/A,-1.17,N/A,-1.8400,N/A,0.4081,N/A,-6.139162,N/A,1.406947,N/A,N/A
2,AAP,158.71,N/A,23.19,N/A,2.9700,N/A,1.1000,N/A,11.249455,N/A,2.466870,N/A,N/A
3,AAPL,133.81,N/A,40.82,N/A,35.1000,N/A,8.6900,N/A,29.581759,N/A,22.672618,N/A,N/A
4,ABBV,106.15,N/A,23.74,N/A,12.5500,N/A,4.6200,N/A,16.257102,N/A,9.651733,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,111.85,N/A,34,N/A,-4.1400,N/A,6.1700,N/A,24.883622,N/A,10.609759,N/A,N/A
501,ZBH,158.08,N/A,-206.82,N/A,2.7500,N/A,4.5500,N/A,26.937422,N/A,7.817424,N/A,N/A
502,ZBRA,388.05,N/A,44.62,N/A,10.9000,N/A,4.9300,N/A,30.881675,N/A,11.372990,N/A,N/A
503,ZION,44.20,N/A,18.42,N/A,1.0436,N/A,2.4400,N/A,10.773189,N/A,2.850040,N/A,N/A


## Dealing With Missing Data in Our DataFrame
Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using.

You can use pandas' isnull method to identify missing data:

In [13]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,214.70,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,235.57,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.00,N/A,9.78,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,26.69,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,51.26,N/A,14.84,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,56.71,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,28.90,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,151.98,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
194,FTI,9.85,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,1814.61,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


## Dealing with missing data is an important topic in data science.
There are two main approaches:
- Drop missing data from the data set (pandas' dropna method is useful here)
- Replace missing data with a new value (pandas' fillna method is useful here)

In this tutorial, we will replace missing data with the average non-NaN data point from that column.

Here is the code to do this:

In [14]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [15]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles
We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:
- Price-to-earnings ratio
- Price-to-book ratio
- Price-to-sales ratio
- EV/EBITDA
- EV/GP

Here's how we'll do this:

In [16]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100
#
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,118.32,N/A,53.76,0.853465,7.2600,0.715842,7.0000,0.798020,33.889454,0.847525,13.187126,0.798020,N/A
1,AAL,16.95,N/A,-1.17,0.140594,-1.8400,0.045545,0.4081,0.029703,-6.139162,0.027723,1.406947,0.053465,N/A
2,AAP,158.71,N/A,23.19,0.449505,2.9700,0.413861,1.1000,0.163366,11.249455,0.285149,2.466870,0.126733,N/A
3,AAPL,133.81,N/A,40.82,0.744554,35.1000,0.962376,8.6900,0.861386,29.581759,0.821782,22.672618,0.960396,N/A
4,ABBV,106.15,N/A,23.74,0.457426,12.5500,0.853465,4.6200,0.628713,16.257102,0.500990,9.651733,0.653465,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,111.85,N/A,34.00,0.673267,-4.1400,0.041584,6.1700,0.764356,24.883622,0.754455,10.609759,0.691089,N/A
501,ZBH,158.08,N/A,-206.82,0.019802,2.7500,0.388119,4.5500,0.623762,26.937422,0.788119,7.817424,0.506931,N/A
502,ZBRA,388.05,N/A,44.62,0.776238,10.9000,0.831683,4.9300,0.701980,30.881675,0.833663,11.372990,0.718812,N/A
503,ZION,44.20,N/A,18.42,0.360396,1.0436,0.108911,2.4400,0.408911,10.773189,0.255446,2.850040,0.150495,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [17]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,118.32,N/A,53.76,0.853465,7.2600,0.715842,7.0000,0.798020,33.889454,0.847525,13.187126,0.798020,0.802574
1,AAL,16.95,N/A,-1.17,0.140594,-1.8400,0.045545,0.4081,0.029703,-6.139162,0.027723,1.406947,0.053465,0.059406
2,AAP,158.71,N/A,23.19,0.449505,2.9700,0.413861,1.1000,0.163366,11.249455,0.285149,2.466870,0.126733,0.287723
3,AAPL,133.81,N/A,40.82,0.744554,35.1000,0.962376,8.6900,0.861386,29.581759,0.821782,22.672618,0.960396,0.870099
4,ABBV,106.15,N/A,23.74,0.457426,12.5500,0.853465,4.6200,0.628713,16.257102,0.500990,9.651733,0.653465,0.618812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,111.85,N/A,34.00,0.673267,-4.1400,0.041584,6.1700,0.764356,24.883622,0.754455,10.609759,0.691089,0.584950
501,ZBH,158.08,N/A,-206.82,0.019802,2.7500,0.388119,4.5500,0.623762,26.937422,0.788119,7.817424,0.506931,0.465347
502,ZBRA,388.05,N/A,44.62,0.776238,10.9000,0.831683,4.9300,0.701980,30.881675,0.833663,11.372990,0.718812,0.772475
503,ZION,44.20,N/A,18.42,0.360396,1.0436,0.108911,2.4400,0.408911,10.773189,0.255446,2.850040,0.150495,0.256832


## Selecting the 50 Best Value Stocks¶
As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [18]:
rv_dataframe.sort_values('RV Score', ascending = False, inplace=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,NOW,569.72,N/A,159.80,0.980198,44.87,0.968317,28.00,0.996040,222.931856,0.998020,34.051575,0.996040,0.987723
1,PAYC,451.65,N/A,153.74,0.976238,44.98,0.970297,34.12,0.998020,92.310173,0.982178,36.674725,1.000000,0.985347
2,ADSK,301.45,N/A,162.33,0.984158,986.44,1.000000,18.71,0.973267,98.243112,0.990099,19.416553,0.928713,0.975248
3,DXCM,364.92,N/A,153.28,0.974257,23.77,0.930693,19.35,0.980198,96.063700,0.988119,28.104839,0.980198,0.970693
4,AMD,92.70,N/A,127.10,0.966337,28.80,0.954455,13.46,0.946535,74.724621,0.974257,28.722443,0.984158,0.965149
5,IDXX,504.98,N/A,95.76,0.944554,157.60,0.988119,17.34,0.968317,63.185805,0.924752,30.545811,0.988119,0.962772
6,NVDA,532.14,N/A,89.42,0.936634,21.61,0.922772,22.35,0.990099,64.119722,0.928713,34.797583,0.998020,0.955248
7,MKTX,588.32,N/A,79.99,0.920792,24.16,0.934653,34.35,1.000000,57.692004,0.912871,32.588517,0.994059,0.952475
8,ISRG,843.50,N/A,96.29,0.948515,10.81,0.828713,22.80,0.992079,69.603905,0.970297,32.075976,0.992079,0.946337
9,MA,359.80,N/A,55.71,0.867327,62.40,0.980198,23.23,0.994059,39.269529,0.877228,22.848536,0.962376,0.936238


## Calculating the Number of Shares to Buy
We'll use the portfolio_input function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [19]:
portfolio_input()

Enter the value of your portfolio:10000000


In [20]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:1719: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,NOW,569.72,351,159.80,0.980198,44.87,0.968317,28.00,0.996040,222.931856,0.998020,34.051575,0.996040,0.987723
1,PAYC,451.65,442,153.74,0.976238,44.98,0.970297,34.12,0.998020,92.310173,0.982178,36.674725,1.000000,0.985347
2,ADSK,301.45,663,162.33,0.984158,986.44,1.000000,18.71,0.973267,98.243112,0.990099,19.416553,0.928713,0.975248
3,DXCM,364.92,548,153.28,0.974257,23.77,0.930693,19.35,0.980198,96.063700,0.988119,28.104839,0.980198,0.970693
4,AMD,92.70,2157,127.10,0.966337,28.80,0.954455,13.46,0.946535,74.724621,0.974257,28.722443,0.984158,0.965149
5,IDXX,504.98,396,95.76,0.944554,157.60,0.988119,17.34,0.968317,63.185805,0.924752,30.545811,0.988119,0.962772
6,NVDA,532.14,375,89.42,0.936634,21.61,0.922772,22.35,0.990099,64.119722,0.928713,34.797583,0.998020,0.955248
7,MKTX,588.32,339,79.99,0.920792,24.16,0.934653,34.35,1.000000,57.692004,0.912871,32.588517,0.994059,0.952475
8,ISRG,843.50,237,96.29,0.948515,10.81,0.828713,22.80,0.992079,69.603905,0.970297,32.075976,0.992079,0.946337
9,MA,359.80,555,55.71,0.867327,62.40,0.980198,23.23,0.994059,39.269529,0.877228,22.848536,0.962376,0.936238


## Formatting Our Excel Output
We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [21]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:
- String format for tickers
- \$XX.XX format for stock prices
- \$XX,XXX format for market capitalization
- Integer format for the number of shares to purchase
-Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [22]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [23]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [24]:
writer.save()